In [1]:
import numpy as np
import pandas as pd
from pandas import read_csv
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model
from keras.layers import Bidirectional

Using TensorFlow backend.


In [2]:
dataset = read_csv('data_earthquake_2000-2017.csv',encoding='latin1', sep=';')

In [13]:
dataset.head()

,Año,Mes,Día,Hora,Minuto,Segundo,Fecha,Latitud,Longitud,Prof.,Magn,Date,Horario,Dtime,time,diff
0,2000,1,2,3,42,12.7,36527,-33.410,-70.880,81.9,2.5,36527,0.154314,36527.15431,1.154314,0.000000
1,2000,1,2,5,3,13.1,36527,-33.787,-71.339,45.8,3.7,36527,0.210568,36527.21057,1.210568,0.056255
2,2000,1,2,18,55,55.0,36527,-31.535,-71.318,35.7,3.8,36527,0.788831,36527.78883,1.788831,0.578263
3,2000,1,2,19,36,25.8,36527,-32.446,-70.923,95.2,2.7,36527,0.816965,36527.81697,1.816965,0.028134
4,2000,1,3,2,41,41.7,36528,-32.986,-71.348,41.8,3.6,36528,0.112288,36528.11229,2.112288,0.295323


In [80]:
len(pd.unique(dataset['Date']))


6533

In [3]:
def build_data(dataset):
    freq = []
    max_magn = 0
    magns = []
    dates = []

    for i,(key,g) in enumerate(dataset.groupby(['Date'])):
        count = 0
        for magn, prof in zip(g['Magn'],g['Prof.']):
            if magn >= 3.5 and prof<=60:
                count+=1
                if magn > max_magn:
                    max_magn = magn
                
        if count != 0: 
            freq.append(count)
            magns.append(max_magn)
            dates.append(str(g['Día'].values)+str(g['Mes'].values)+str(g['Año'].values))
        max_magn = 0
    
    if len(magns) != len(freq):
        print('doh!')
        
    d = {'Freq':freq,'Max Magn':magns}    
    return pd.DataFrame(d)


def build_time_series(data,timestep,y_col):
    
    dim_0 = data.shape[0] - timestep
    dim_1 = data.shape[1] ##only apply with more attributes data.shape[0]
    x = np.zeros((dim_0, timestep,dim_1))
    y = np.zeros((dim_0,))
    for i in range(dim_0):
        x[i] = data[i:timestep+i]
        y[i] = data[timestep+i,y_col]
    return x,y

def scale(data,scaler):
    data['Max Magn'] = cs.fit_transform(data[['Max Magn']])
    return data

In [4]:
#iquique_lat= dataset[ dataset['Latitud'] >= -20.2132607 ]
#iquique_log = dataset[dataset['Longitud'] <= -70.1502686]
cs = MinMaxScaler()
#iquique = pd.concat([iquique_lat,iquique_log]).drop_duplicates(keep=False)
##print(iquique)
iquique = build_data(dataset.query('Latitud >= -20.2132607 and Longitud  <= -70.1502686'))
coquimbo = dataset.query('-20.2132607  <= Latitud >= -29.9533195  and -71.3394699 <= Longitud >= -70.1502686')
concepcion = dataset.query('-36.8269882 <= Latitud and  Longitud >= -73.0497665 ')
#coquimbo_lat = dataset[ dataset['Latitud'] >= -29.9533195 <=  -20.2132607]
#coquimbo_log = dataset[dataset['Longitud'] <= -71.3394699 and >= -70.1502686 ]
#coquimbo = pd.concat([coquimbo_lat,coquimbo_log]).drop_duplicates(keep=False)

#y_iq = get_max_magn(iquique,20)

#x_iq = generate_time_series(freq_iq,20)
print(iquique.head())
iquique = scale(iquique,cs)
print(iquique.head())
X,y = build_time_series(iquique.values,30,1)
X.shape

   Freq  Max Magn
0     1       5.0
1     1       5.8
2     1       4.3
3     1       4.6
4     1       5.6
   Freq  Max Magn
0     1  0.319149
1     1  0.489362
2     1  0.170213
3     1  0.234043
4     1  0.446809


(389, 30, 2)

In [12]:
lstm_model = Sequential()
lstm_model.add(LSTM(100, batch_input_shape=(BATCH_SIZE, TIME_STEPS, x_t.shape[2]), dropout=0.0, recurrent_dropout=0.0, stateful=True,     kernel_initializer='random_uniform'))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(20,activation='relu'))
lstm_model.add(Dense(1,activation='sigmoid'))
optimizer = optimizers.RMSprop(lr=lr)
lstm_model.compile(loss='mean_squared_error', optimizer=optimizer)

216


In [68]:
n = len(series)
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n,1)))
model.add(RepeatVector(n))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')



In [65]:
model.fit(series, series, epochs=300, verbose=0)
plot_model(model, show_shapes=True, to_file='reconstruct_lstm_autoencoder.png')
yhat = model.predict(series, verbose=0)
print(yhat[0,:,0])


You can find the C code in this temporary file: /var/folders/ds/0m5xqyr14q3dzywwn54tq2q80000gn/T/theano_compilation_error_v9dy14h5


Exception: ('The following error happened while compiling the node', forall_inplace,cpu,scan_fn}(Elemwise{maximum,no_inplace}.0, Subtensor{int64:int64:int8}.0, IncSubtensor{InplaceSet;:int64:}.0, IncSubtensor{InplaceSet;:int64:}.0, Elemwise{Maximum}[(0, 0)].0, Subtensor{::, int64:int64:}.0, InplaceDimShuffle{x,0}.0, Subtensor{::, int64:int64:}.0, Subtensor{::, :int64:}.0, InplaceDimShuffle{x,0}.0, Subtensor{::, :int64:}.0, Subtensor{::, int64:int64:}.0, InplaceDimShuffle{x,0}.0, Subtensor{::, int64:int64:}.0, Subtensor{::, int64::}.0, InplaceDimShuffle{x,0}.0, Subtensor{::, int64::}.0), '\n', "Compilation failed (return status=1): /Users/stroncod/.theano/compiledir_Darwin-16.7.0-x86_64-i386-64bit-i386-3.7.0-64/scan_perform/mod.cpp:11860:65: error: too many arguments to function call, expected 3, have 4.     return (*((__Pyx_PyCFunctionFast)meth)) (self, args, nargs, NULL);.            ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~                     ^~~~. /Applications/Xcode.app/Contents/Developer/